# Using Intrinsics on OpenAI-Compatible Inference Backends

This notebook demonstrates how to use the shared input and output processing code for
intrinsics when performing model inference with an OpenAI-compatible backend such as
vLLM.

In [ ]:
# Imports go in this cell
import os
import openai
import json
import granite_common

## Constants

Change the value of the constants `intrinsic_name` and `base_model` in the cell that 
follows to change which intrinsic will be demonstrated in the remainder of this notebook. Other constants will automatically adjust accordingly.

You may also need to adjust the values of `openai_base_url` and `openai_api_key` to correspond to the location of the server where you are hosting the LoRA adapter.

In [ ]:
# Change the following constants to select a different intrinsic for the remainder
# of this notebook.
intrinsic_name = "citations"
base_model_name = "granite-3.3-8b-instruct"
intrinsics_repo_name = "ibm-granite/rag-intrinsics-lib"
use_alora = False

# Change the following two constants as needed to reflect the location of the inference
# server.
openai_base_url = "http://localhost:55555/v1/"
openai_api_key = "rag_intrinsics_1234"


#######################################################################################
# The code below adjusts the remaining constants according to the chosen intrinsic.

TESTDATA_DIR = "../tests/granite_common/intrinsics/rag/testdata"
KNOWN_INTRINSICS = [
    "answerability",
    "answer_relevance_classifier",
    "answer_relevance_rewriter",
    "citations",
    "context_relevance",
    "hallucination_detection",
    "query_rewrite",
    "requirement_check",
    "uncertainty",
]
INTRINSICS_WITH_LOCAL_YAML_FILES = []

io_yaml_file = None  # None -> load from Hugging Face Hub
request_json_file = f"{TESTDATA_DIR}/input_json/{intrinsic_name}.json"

# Include local JSON file with arguments if that file is present.
maybe_arg_file = f"{TESTDATA_DIR}/input_args/{intrinsic_name}.json"
arg_file = maybe_arg_file if os.path.exists(maybe_arg_file) else None

# Selectively override defaults
if intrinsic_name == "answerability":
    request_json_file = f"{TESTDATA_DIR}/input_json/answerable.json"
elif intrinsic_name in INTRINSICS_WITH_LOCAL_YAML_FILES:
    # Some io.yaml files not yet delivered to Hugging Face Hub
    io_yaml_file = f"{TESTDATA_DIR}/input_yaml/{intrinsic_name}.yaml"
elif intrinsic_name not in KNOWN_INTRINSICS:
    raise ValueError(f"Unrecognized intrinsic name '{intrinsic_name}'")

if io_yaml_file is None:
    # Fetch IO configuration file from Hugging Face Hub
    io_yaml_file = granite_common.intrinsics.obtain_io_yaml(
        intrinsic_name, base_model_name, intrinsics_repo_name, alora=use_alora
    )

# Print the variables we just set
print(f"{io_yaml_file=}")
print(f"{request_json_file=}")
print(f"{arg_file=}")

## Instantiate the input and output processing classes

The constructors for the classes `IntrinsicsRewriter` and `IntrinsicsResultProcessor`
serve as factory methods to produce input and output processors, respectively, for 
a given intrinsic.

In [ ]:
print(
    f"Instantiating input and output processing from configuration file:\n"
    f"{io_yaml_file}"
)

rewriter = granite_common.IntrinsicsRewriter(config_file=io_yaml_file)
result_processor = granite_common.IntrinsicsResultProcessor(config_file=io_yaml_file)

## Perform input processing

The cells that follow load an example OpenAI-compatible chat completion request from
a local file, then show how to apply input processing to the request.

In [ ]:
# Read original request from the appropriate file
print(f"Loading request data from {request_json_file}")
with open(request_json_file, encoding="utf-8") as f:
    request_json_str = f.read()
request_json = json.loads(request_json_str)

# Some parameters like model name aren't kept in the JSON files that we use for testing.
# Apply appropriate values for those parameters.
request_json["model"] = intrinsic_name
request_json["temperature"] = 0.0

print("Original request:")
print(json.dumps(request_json, indent=2))

In [ ]:
# Some intrinsics take one or more additional arguments besides the target chat
# completion request. Load the additional arguments from a file if that is the case.
intrinsic_kwargs = {}
if arg_file is not None:
    with open(arg_file, encoding="utf8") as file:
        intrinsic_kwargs = json.load(file)
    print(f"Using additional arguments:\n{intrinsic_kwargs}")

In [ ]:
# Run request through input processing.
rewritten_request = rewriter.transform(request_json, **intrinsic_kwargs)

print("Request after input processing:")
print(rewritten_request.model_dump_json(indent=2))

## Run inference

Passing a request through the input processing `RagAgentLibRewriter.transform()` 
turns the request into something that can be sent directly to an OpenAI-compatible
inference endpoint for the intrinsic.

The cells that follow show how to perform inference using the chat completions API.

To run these cells, you'll need to start a server such as vLLM that serves the intrinsic
under the appropriate model name at the base URL specified by `openai_base_url` in the constants cell above.

In [ ]:
# Connect to the local inference server
client = openai.OpenAI(base_url=openai_base_url, api_key=openai_api_key)

In [ ]:
# Pass our rewritten request directly to `chat.completions.create()`
chat_completion = client.chat.completions.create(**rewritten_request.model_dump())

print("Immediately after low-level inference, first completion is:")
print(chat_completion.choices[0].model_dump_json(indent=2))

In [ ]:
# Print the raw JSON for the convenience of developers who might need that data.
print(chat_completion.model_dump_json(indent=2))

## Post-process inference results

The raw output of some intrinsics requires some additional postprocessing to turn it 
into a form that is easy to consume in an application. This postprocessing occurs in
the method `RagAgentLibResultProcessor.transform()`. 

The cells that follow show how to use this method to transform the raw output of the
`chat.completions.create()` API call into the intrinsic's application-level output
value.

By convention, this application-level output value is returned in the same format as a
chat completions request result.

In [ ]:
processed_chat_completion = result_processor.transform(
    chat_completion, rewritten_request
)

print("After post-processing, first completion is:")
print(processed_chat_completion.choices[0].model_dump_json(indent=2))

In [ ]:
# Verify that the contents of the completion is valid JSON and pretty-print the JSON.
parsed_contents = json.loads(processed_chat_completion.choices[0].message.content)
print("JSON contents of first completion:")
print(json.dumps(parsed_contents, indent=2))

In [ ]:
# Full JSON output of post-processing
print(processed_chat_completion.model_dump_json(indent=2))